In [7]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [17]:
world_df = pd.read_csv('world.csv')
life_df = pd.read_csv('life.csv')

world_df['Year'] = world_df['Year'].astype(str)
life_df['Year'] = life_df['Year'].astype(str)

merged_df = pd.merge(world_df, life_df, on=['CountryName', 'CountryCode', 'Year'])
merged_df.sort_values(by=['CountryName'], inplace=True)
merged_df.dropna(subset=['CountryName'],inplace=True)
print(merged_df.head())

           CountryName  Year CountryCode  \
0          Afghanistan  2016         AFG   
1              Albania  2016         ALB   
2              Algeria  2016         DZA   
3               Angola  2016         AGO   
4  Antigua and Barbuda  2016         ATG   

  Access to electricity (% of population) [EG.ELC.ACCS.ZS]  \
0                                               97.7         
1                                                100         
2                                         99.9923172         
3                                        40.62452698         
4                                                100         

  Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]  \
0                                        465.8415828                          
1                                        3776.111694                          
2                                        3284.470252                          
3                                         1949

In [ ]:
#task 1A
X=merged_df.drop(columns=['CountryName', 'CountryCode', 'Year', 'Life expectancy at birth (years)'])
y=merged_df['Life expectancy at birth (years)']
#训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=200)  #测试训练三七分
X_train=X_train.replace('..',np.nan)
X_test=X_test.replace('..',np.nan)  #将..替换为nan
#用中位数填充nan
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
#标准化
scaler=StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
#knn分类器
knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_7 = KNeighborsClassifier(n_neighbors=7)
knn_3.fit(X_train_scaled, y_train)
knn_7.fit(X_train_scaled, y_train)
#决策树分类器
dt=DecisionTreeClassifier(max_depth=3)
dt.fit(X_train_scaled, y_train)
#测试分类器
y_pred_knn_3 = knn_3.predict(X_test_scaled)
y_pred_knn_7 = knn_7.predict(X_test_scaled)
y_pred_dt = dt.predict(X_test_scaled)
#计算准确率
accuracy_knn_3 = accuracy_score(y_test, y_pred_knn_3)
accuracy_knn_7 = accuracy_score(y_test, y_pred_knn_7)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f'k-NN (k=3) 的准确率:{accuracy_knn_3}')
print(f'k-NN (k=7) 的准确率:{accuracy_knn_7}')
print(f'决策树的准确率:{accuracy_dt}')

k-NN (k=3) 的准确率:0.6041666666666666
k-NN (k=7) 的准确率:0.6041666666666666
决策树的准确率:0.7708333333333334
